# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""13117902318007…",1.2927e19,"""18222093151955…","""../../../../da…","""17610899682975…","""95635181211216…",false
"""87050172472223…",1.4856e19,"""63278546923972…","""../../../../da…","""17610899682975…","""21200423566337…",true
"""26032848387461…",5.6458e17,"""10960768903630…","""../../../../da…","""14637267072216…","""95635181211216…",false
"""12973445202690…",4.0723e18,"""63278546923972…","""../../../../da…","""22144418934571…","""95635181211216…",false
"""16615297116411…",1.4677e19,"""18222093151955…","""../../../../da…","""22144418934571…","""15395978230056…",true
"""13122921046063…",1.6330e19,"""10960768903630…","""../../../../da…","""14964891934801…",null,false
"""75772816642685…",8.7675e18,"""10960768903630…","""../../../../da…","""22144418934571…","""15395978230056…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_MaritalStatus': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""11093353965703…",1.2927e19,"""51722480423986…","""../../../../da…","""FirstMortgage3…","""94438586201907…","""Rejected"""
"""14461183009423…",1.4856e19,"""13185704252364…","""../../../../da…","""FirstMortgage3…","""18050394246542…","""Accepted"""
"""55883827594872…",5.6458e17,"""86874672239904…","""../../../../da…","""MoneyMarketSav…","""94438586201907…","""Rejected"""
"""18986324317992…",4.0723e18,"""13185704252364…","""../../../../da…","""BasicChecking""","""94438586201907…","""Rejected"""
"""78460097197768…",1.4677e19,"""51722480423986…","""../../../../da…","""BasicChecking""","""16756569768293…","""Accepted"""
"""11617208914095…",1.6330e19,"""86874672239904…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""58458120557961…",8.7675e18,"""86874672239904…","""../../../../da…","""BasicChecking""","""16756569768293…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""13139438789826…",1.2927e19,"""65988428504416…","""../../../../da…","""FirstMortgage3…","""96284816283175…","""Rejected"""
"""16270209014305…",1.4856e19,"""10924904748511…","""../../../../da…","""FirstMortgage3…","""90112247031953…","""Accepted"""
"""11897107182835…",5.6458e17,"""61762951929808…","""../../../../da…","""MoneyMarketSav…","""96284816283175…","""Rejected"""
"""14963982321815…",4.0723e18,"""10924904748511…","""../../../../da…","""BasicChecking""","""96284816283175…","""Rejected"""
"""14044326100161…",1.4677e19,"""65988428504416…","""../../../../da…","""BasicChecking""","""15760330121842…","""Accepted"""
"""15875071901847…",1.6330e19,"""61762951929808…","""../../../../da…","""UPlusFinPerson…",null,"""Rejected"""
"""13084237890649…",8.7675e18,"""61762951929808…","""../../../../da…","""BasicChecking""","""15760330121842…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_CLV=PREDICTOR_1
Customer_MaritalStatus=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""22207803884381…",1.2927e19,"""34777341596363…","""FirstMortgage3…","""24340305048576…","""Rejected"""
"""16872808884801…",1.4856e19,"""42735986120920…","""FirstMortgage3…","""44133409117356…","""Accepted"""
"""27716015464166…",5.6458e17,"""15827506130720…","""MoneyMarketSav…","""24340305048576…","""Rejected"""
"""21304752393614…",4.0723e18,"""42735986120920…","""BasicChecking""","""24340305048576…","""Rejected"""
"""14539182554275…",1.4677e19,"""34777341596363…","""BasicChecking""","""11233976508865…","""Accepted"""
"""12999309520606…",1.6330e19,"""15827506130720…","""UPlusFinPerson…",null,"""Rejected"""
"""15192350919805…",8.7675e18,"""15827506130720…","""BasicChecking""","""11233976508865…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'